#### Agents

Very closely related to the concept of Chains, Agents are also used to execute multiple steps using an LLM. A major difference being the way multiple actions are executed by Chains and Agents. In case of Agents, the backend uses an LLM to decide over what action to take (of all options/tools available), hence flexible while in case of Chains, a hardcoded sequence of steps are taken. Also, agents are more diverse and can integrate with multiple, different tools.

You have an agent that has access to
* Google search
* Maths-Calculator
* A retriever for some external document

* When you run a query, the LLM will decide which tool to use depending on the query or maybe it won’t use any tool if the query doesn’t require it like ‘Hello LLM’. But in case of a chain,

* You might not be able to provide multiple, diverse functionality with pre-defined chains in Langchain.

* Even if you do, you need to hardcode the whole code and sequence of steps. So even if the query don’t require any tool, it will use it (& may throw an error as well due to lack of compatibility). Hence no flexibility

* So, if you want a more flexible, dynamic app, Agents are a better option but if your use case is constant, chains can be used.

In [ ]:
# Agent is like a Chatbot which can do specific tasks, they can interact with api, tools

In [ ]:
#!pip install langchain openai google-search-result

In [1]:
from langchain.agents import Tool
from langchain.agents import AgentType
from langchain.memory import ConversationBufferMemory
from langchain.llms import OpenAI
from langchain.utilities import SerpAPIWrapper
from langchain.agents import initialize_agent

In [2]:
# Set your API key : https://platform.openai.com/account/api-keys
# store the API key in Environment variable : https://networkdirection.net/python/resources/env-variable/
import os
api_key = os.environ.get('OpenAI_API_Key')
serp_key = os.environ.get('serp_api')

In [9]:
#You need to get serp_api and openai_api for this.
search = SerpAPIWrapper(serpapi_api_key=serp_key)
llm=OpenAI(openai_api_key=api_key)

tools = [
    Tool(
        name = "Current Search",
        func=search.run,
        description="useful for when you need to answer questions about current events or the current state of the world"

    ),]
search_me = "Explain what happend in G20 meeting, 2023 that happened in Delhi?"

In [6]:
agent_chain = initialize_agent(tools, llm, agent=AgentType.CONVERSATIONAL_REACT_DESCRIPTION)

In [12]:
out = agent_chain({"input": search_me, "chat_history": []})
print(out)

{'input': 'Explain what happend in G20 meeting, 2023 that happened in Delhi?', 'chat_history': [], 'output': 'The G20 meeting in 2023, which was held in Delhi, India, was focused on the global economic recovery and strengthening international cooperation. It was attended by the heads of states from the G20 countries, as well as representatives of international organizations. The meeting discussed issues related to climate change, global trade, and the digital economy, as well as other topics of global importance. The participants also agreed on a new framework for global financial regulation.'}


In [ ]:
# refer : https://github.com/langchain-ai/langchain/issues/3106

#### How to build Langchain agents?

In [ ]:
# Second example: 

In [ ]:
# pip install langchain openai arxiv

In [13]:
from langchain.agents import Tool, ZeroShotAgent, AgentExecutor
from langchain import OpenAI, LLMChain 
from langchain.memory import ConversationBufferMemory
from langchain.tools.arxiv.tool import ArxivQueryRun
from langchain.tools.shell.tool import ShellTool

In [14]:
import langchain.tools as lt
help(lt)
# if we create an agent it can interact with all the tools below to do specific tasks

Help on package langchain.tools in langchain:

NAME
    langchain.tools - Core toolkit implementations.

PACKAGE CONTENTS
    arxiv (package)
    azure_cognitive_services (package)
    base
    bing_search (package)
    brave_search (package)
    convert_to_openai
    ddg_search (package)
    file_management (package)
    gmail (package)
    google_places (package)
    google_search (package)
    google_serper (package)
    graphql (package)
    human (package)
    ifttt
    interaction (package)
    jira (package)
    json (package)
    metaphor_search (package)
    openapi (package)
    openweathermap (package)
    playwright (package)
    plugin
    powerbi (package)
    pubmed (package)
    python (package)
    requests (package)
    scenexplain (package)
    searx_search (package)
    shell (package)
    sleep (package)
    spark_sql (package)
    sql_database (package)
    steamship_image_generation (package)
    vectorstore (package)
    wikipedia (package)
    wolfram_alpha (pa

In [ ]:
# arxiv : what is happening around latest research papers, arxiv query run tool
# shell tool : will be doing it in a shell

In [15]:
# tool list
# we can use multiple tools for multiple tasks
papers = ArxivQueryRun()
shell = ShellTool()

tools = [
    Tool(
        name="papers",
        func=papers.run,
        description="useful for when you need to answer about research paper",

        ),
    Tool(
        name="shell",
        func=shell.run,
        description ="useful for shell scripting and related stuff",
    )
]

In [16]:
# Designing a prompt
prefix = """Have a conversation with a human, answering the following questions as best you can. You have access to the following tools: """
suffix = """Begin!"

{chat_history}
Question: {input}
{agent_scratchpad}"""

# chat_history : is to provide memory to the agent 
# input : prompt
# agent scratchpad : what the agent is actually doing in the background
prompt = ZeroShotAgent.create_prompt(
    tools,
    prefix=prefix,
    suffix=suffix,
    input_variables=["input", "chat_history", "agent_scratchpad"],

)
# ZeroShotAgent : agent which has no examples to learn from the particular tools
memory = ConversationBufferMemory(memory_key="chat_history")
# provide chat history to the agent, it will remember the response of the previous questions

In [17]:
# Create a LLM agent:
llm_chain = LLMChain(llm=OpenAI(temperature=0, openai_api_key=api_key), prompt=prompt)
agent = ZeroShotAgent(llm_chain=llm_chain, tools=tools, verbose=True)
# creating a final chain:
agent_chain = AgentExecutor.from_agent_and_tools(
    agent= agent, tools=tools, verbose = True, memory=memory
)

In [18]:
agent_chain.run(input="Suggest the latest developments around LLMs?")



> Entering new  chain...
Thought: I should research the latest developments in LLMs.
Action: papers
Action Input: Latest developments in LLMs
Observation: Published: 2023-10-09
Title: A Survey of Large Language Models for Healthcare: from Data, Technology, and Applications to Accountability and Ethics
Authors: Kai He, Rui Mao, Qika Lin, Yucheng Ruan, Xiang Lan, Mengling Feng, Erik Cambria
Summary: The utilization of large language models (LLMs) in the Healthcare domain has
generated both excitement and concern due to their ability to effectively
respond to freetext queries with certain professional knowledge. This survey
outlines the capabilities of the currently developed LLMs for Healthcare and
explicates their development process, with the aim of providing an overview of
the development roadmap from traditional Pretrained Language Models (PLMs) to
LLMs. Specifically, we first explore the potential of LLMs to enhance the
efficiency and effectiveness of various Healthcare applicatio

'The latest developments in LLMs include a survey of large language models for healthcare, a new qualitative approach for code LLM evaluation, and enhancing pipeline-based conversational agents with large language models.'

In [ ]:
# we won't get an answer like this if we use ChatGpt 3.5

In [ ]:
# using the shell tool:

In [21]:
agent_chain.run(input= " code to print 'hello world' and execute hello.py")



> Entering new  chain...
Thought: I need to find the code to print 'Hello World' and execute hello.py
Action: shell
Action Input: echo "print('Hello World')" > hello.py followed by python hello.py
Observation: 
Thought: I now know the final answer
Final Answer: echo "print('Hello World')" > hello.py followed by python hello.py

> Finished chain.


'echo "print(\'Hello World\')" > hello.py followed by python hello.py'

#### How to create Custom tools for Langchain Agent ?

In [ ]:
#! pip install langchain openai langchain-experimental yfinance

In [1]:
import yfinance as yf
from datetime import datetime, timedelta

In [4]:
def get_current_stock_price(ticker):
    
    """Method to get current stock price"""

    ticker_data = yf.Ticker(ticker)
    recent = ticker_data.history(period="1d")
    return {"price": recent.iloc[0]["Close"], "currency": ticker_data.info["currency"]}

In [5]:
def get_stock_performance(ticker, days):
    """Method to get stock price change in percentage"""
    past_date = datetime.today() - timedelta(days=days)
    ticker_data = yf.Ticker(ticker)
    history = ticker_data.history(start=past_date)
    old_price = history.iloc[0]["Close"]
    current_price = history.iloc[-1]["Close"]
    return {"percent_change" : ((current_price - old_price)/ old_price) * 100}

In [6]:
from typing import Type
from pydantic import BaseModel, Field
from langchain.tools import BaseTool
# for the tool we need two classes - input for the particular tool

class CurrentStockPriceInput(BaseModel):
    """Inputs for get_current_stock_price"""
    ticker: str = Field(description="Ticker symbol of the stock")


In [7]:
class CurrentStockPriceTool(BaseTool):
    # inheriting BaseTool which is coming from langchain.tools. So, we are not using an existing tool with LangChain, we are using custom tool
    name = "get_current_stock_price"
    description = """
        Useful when you want to get current stock price.
        You should enter the stock ticker symbol recognized by the yahoo finance
        """
    args_schema: Type[BaseModel] = CurrentStockPriceInput

    def _run(self, ticker: str):
        price_response = get_current_stock_price(ticker)
        return price_response
    
    def _arun(self, ticker: str):
        raise NotImplementedError("get_current_stock_price does not support async")

In [8]:
class StockPercentChangeInput(BaseModel):
    """Inputs for get_stock_performance"""
    ticker: str = Field(description="Ticker symbol of the stock")
    days: int = Field(description="Timedelta days to get past date from current date")

class StockPerformanceTool(BaseTool):
    name="get_stock_performance"""
    description = """
        Useful when you want to check performance of the stock.
        You should enter the stock ticker symbol recognized by the yahoo finance.
        You should enter days as number of days from today from which performance needs to be checked. 
        output will be the change in the stock price represented as a percentage.
        """
    args_schema: Type[BaseModel] = StockPercentChangeInput

    def _run(self, ticker: str, days: int):
        response = get_stock_performance(ticker,days)
        return response
        
    def _arun(self, ticker: str):
        raise NotImplementedError("get_stock_performance does not support async")


In [9]:
from langchain.agents import AgentType
from langchain.chat_models import ChatOpenAI
from langchain.agents import initialize_agent

In [10]:
llm = ChatOpenAI(openai_api_key=api_key)
#using the custom tools
tools = [CurrentStockPriceTool(), StockPerformanceTool()]
agent = initialize_agent(tools, llm, agent=AgentType.OPENAI_FUNCTIONS, verbose=True)


In [11]:
agent.run("What is the current price of Microsoft stock? How it has performed over past 6 months?")



> Entering new AgentExecutor chain...

Invoking: `get_current_stock_price` with `{'ticker': 'MSFT'}`


{'price': 329.32000732421875, 'currency': 'USD'}
Invoking: `get_stock_performance` with `{'ticker': 'MSFT', 'days': 180}`


{'percent_change': 11.974368403663815}The current price of Microsoft stock is $329.32 USD. 

Over the past 6 months, Microsoft stock has performed well with a 11.97% increase in its price.

> Finished chain.


'The current price of Microsoft stock is $329.32 USD. \n\nOver the past 6 months, Microsoft stock has performed well with a 11.97% increase in its price.'

#### Callbacks

LangChain offers a callback system enabling us to integrate with different phases of our Language Model application, serving purposes such as logging, monitoring, streaming, and other essential functions.

In [3]:
from langchain.callbacks import StdOutCallbackHandler
from langchain.chains import LLMChain
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate

In [11]:
# code snippet explains how to log every event using the Standard Output callback
handler = StdOutCallbackHandler()
llm = OpenAI()
prompt = PromptTemplate.from_template("Write {number} lines about {animal} ")


In [12]:
# Constructor callback: First, let's explicitly set the StdOutCallbackHandler when initializing our chain
chain = LLMChain(llm=llm, prompt=prompt, callbacks=[handler])
chain.run(number=3,animal='tiger')



> Entering new LLMChain chain...
Prompt after formatting:
Write 3 lines about tiger 

> Finished chain.


'\n\n1. Tigers are the biggest species of cats in the world. \n2. They are native to Asia and are known for their striped coats. \n3. Tigers are apex predators, relying on their strength and power to hunt and survive.'

#### End